In [1]:
# --- 1. ライブラリのインポート ---
import os
import pandas as pd
import requests
import re
from time import sleep
from datetime import datetime
from Bio import Entrez
from googletrans import Translator
import ipywidgets as widgets
from IPython.display import display

In [2]:
# --- 基本設定 ---

# NCBI Entrezへの登録メールアドレス (必須)
# PubMedのAPIを利用するために、NCBIに自分の身元を伝えるメールアドレスが必要です。
Entrez.email = "gakukibou@gmail.com"  # ★ 必ずご自身のメールアドレスに書き換えてください

TARGET_JOURNALS_WITH_IF = [
    {'name': 'Nature Reviews Drug Discovery', 'if': 120.1}, {'name': 'Nature Reviews Molecular Cell Biology', 'if': 98.6},
    {'name': 'Cancer Cell', 'if': 52.4}, {'name': 'Cell', 'if': 50.5}, {'name': 'Nature', 'if': 48.3},
    {'name': 'Science', 'if': 46.5}, {'name': 'Nature Medicine', 'if': 45.1}, {'name': 'Nature Chemical Biology', 'if': 43.8},
    {'name': 'Nature Structural & Molecular Biology', 'if': 42.1}, {'name': 'Cancer Discovery', 'if': 38.5},
    {'name': 'Molecular Cell', 'if': 37.9}, {'name': 'Journal of Clinical Oncology', 'if': 35.1},
    {'name': 'Immunity', 'if': 32.4}, {'name': 'Genes & Development', 'if': 31.2},
    {'name': 'Annual Review of Biochemistry', 'if': 30.5}, {'name': 'eLife', 'if': 29.8},
    {'name': 'Journal of the American Chemical Society', 'if': 28.7}, {'name': 'Angewandte Chemie International Edition', 'if': 27.5},
    {'name': 'Trends in Pharmacological Sciences', 'if': 26.8}, {'name': 'Proceedings of the National Academy of Sciences', 'if': 25.4},
    {'name': 'Journal of Medicinal Chemistry', 'if': 24.1}
]

In [3]:
# --- 3. UIウィジェットの作成 ---

# ジャーナルのチェックボックスを作成
journal_checkboxes = [
    widgets.Checkbox(value=True, description=f"{journal['name']} (IF: {journal['if']})", indent=False, layout=widgets.Layout(width='95%')) 
    for journal in TARGET_JOURNALS_WITH_IF
]

# その他の入力ウィジェットを作成
topic_input = widgets.Text(value='KRAS AND G12D AND inhibitor', placeholder='研究キーワードを入力', description='キーワード:', layout=widgets.Layout(width='80%'))
start_year_input = widgets.IntText(value=2023, description='開始年:')
end_year_input = widgets.IntText(value=2025, description='終了年:')
max_results_input = widgets.IntText(value=20, description='最大件数:')
csv_path_input = widgets.Text(value='pubmed_論文リスト_2.csv', placeholder='保存ファイル名', description='保存先:', layout=widgets.Layout(width='80%'))


In [4]:
# --- 4. UIの組み立てと表示 ---
journal_layout = widgets.Layout(height='300px', border='1px solid black', padding='5px', overflow_y='scroll')
journal_selection_ui = widgets.VBox(journal_checkboxes, layout=journal_layout)
input_ui = widgets.VBox([
    topic_input,
    widgets.HBox([start_year_input, end_year_input, max_results_input]),
    csv_path_input
])

print("--- トップジャーナル論文検索ツール ---")
print("1. 以下のリストから、検索対象とするジャーナルを選択（チェックを外すと除外）してください。")
display(journal_selection_ui)
print("\n2. 検索条件を入力・確認し、下のセルを実行して検索を開始してください。")
display(input_ui)

--- トップジャーナル論文検索ツール ---
1. 以下のリストから、検索対象とするジャーナルを選択（チェックを外すと除外）してください。



2. 検索条件を入力・確認し、下のセルを実行して検索を開始してください。


In [6]:
# --- ヘルパー関数の定義 ---
def translate_text(text, dest_lang='ja'):
    if not text: return ""
    try:
        translator = Translator(); sleep(1)
        return translator.translate(text, dest=dest_lang).text
    except Exception as e:
        print(f"翻訳中にエラーが発生しました: {e}"); return "(翻訳失敗)"

# --- 1. UIから設定値を読み取り ---
selected_journals = [cb.description.split(' (IF:')[0] for cb in journal_checkboxes if cb.value]
topic_keywords = topic_input.value
start_year = start_year_input.value
end_year = end_year_input.value
max_results = max_results_input.value
csv_file_path = csv_path_input.value

# --- 2. 検索処理の実行 ---
if not topic_keywords or not selected_journals:
    print("キーワードが入力されていないか、ジャーナルが選択されていません。処理を終了します。")
else:
    journal_filter = " OR ".join([f'"{journal}"[Journal]' for journal in selected_journals])
    final_search_term = f"({topic_keywords}) AND ({journal_filter})"
    date_filter = ""
    if start_year and end_year: date_filter = f"({start_year}[Date - Publication] : {end_year}[Date - Publication])"
    elif start_year: date_filter = f"({start_year}[Date - Publication] : {datetime.now().year}[Date - Publication])"
    elif end_year: date_filter = f"(1000[Date - Publication] : {end_year}[Date - Publication])"
    if date_filter:
        final_search_term = f"({final_search_term}) AND {date_filter}"

    print("\n--- 設定内容の確認 ---")
    print(f"検索対象ジャーナル数: {len(selected_journals)}誌")
    print(f"最終検索クエリ (一部): {final_search_term[:200]}...")
    print(f"最大検索数: {max_results}")
    print(f"保存ファイル名: {csv_file_path}")
    print("---------------------\n")
    
    try:
        df_existing = pd.read_csv(csv_file_path); existing_ids = set(df_existing['pmid'].astype(str))
        print(f"既存の論文リスト '{csv_file_path}' を読み込みました。{len(existing_ids)}件が登録済みです。")
    except FileNotFoundError:
        df_existing = pd.DataFrame(); existing_ids = set()
        print(f"新規に論文リスト '{csv_file_path}' を作成します。")

    print(f"PubMedで検索中...")
    handle = Entrez.esearch(db="pubmed", term=final_search_term, retmax=max_results, sort="pub date")
    record = Entrez.read(handle); handle.close()
    id_list = record["IdList"]

    if not id_list:
        print("検索結果が0件でした。")
    else:
        print(f"{len(id_list)}件の論文IDを取得しました。詳細情報を1件ずつ処理します。")
        handle = Entrez.efetch(db="pubmed", id=id_list, rettype="medline", retmode="xml")
        records = Entrez.read(handle)['PubmedArticle']; handle.close()

        new_articles = []
        for record in records:
            citation = record['MedlineCitation']; pmid = str(citation['PMID'])
            if pmid in existing_ids: continue

            print(f"\n--- 新規論文を処理中 (PMID: {pmid}) ---")
            article = citation['Article']
            title = article.get('ArticleTitle', 'N/A')
            journal_info = article.get('Journal', {}); journal = journal_info.get('Title', 'N/A')
            pub_date = journal_info.get('JournalIssue', {}).get('PubDate', {})
            pub_year = pub_date.get('Year', pub_date.get('MedlineDate', 'N/A').split(' ')[0])
            author_list = article.get('AuthorList', [])
            authors = ", ".join([f"{author.get('LastName', '')} {author.get('ForeName', '')}".strip() for author in author_list])
            abstract_en = " ".join(article.get('Abstract', {}).get('AbstractText', []))
            article_info = {'pmid': pmid, 'title': title, 'pub_year': pub_year, 'journal': journal, 'authors': authors, 'url': f"https://pubmed.ncbi.nlm.nih.gov/{pmid}/"}
            print("要約を翻訳中...")
            article_info['abstract_ja'] = translate_text(abstract_en)
            new_articles.append(article_info)
            sleep(1)

        if new_articles:
            print(f"\n{len(new_articles)}件の新規論文が見つかりました。CSVファイルに追加します。")
            df_new = pd.DataFrame(new_articles)
            column_order = ['pmid', 'title', 'pub_year', 'journal', 'authors', 'url', 'abstract_ja']
            df_new = df_new.reindex(columns=column_order)
            df_all = pd.concat([df_existing, df_new], ignore_index=True)
            df_all.to_csv(csv_file_path, index=False, encoding='utf-8-sig')
            print(f"'{csv_file_path}' に保存しました。")
        else:
            print("\n新しい論文は見つかりませんでした（既に登録済みの論文はスキップされます）。")


--- 設定内容の確認 ---
検索対象ジャーナル数: 21誌
最終検索クエリ (一部): ((KRAS AND G12D AND MRTX) AND ("Nature Reviews Drug Discovery"[Journal] OR "Nature Reviews Molecular Cell Biology"[Journal] OR "Cancer Cell"[Journal] OR "Cell"[Journal] OR "Nature"[Journal] OR "Scienc...
最大検索数: 5
保存ファイル名: pubmed_論文リスト_3.csv
---------------------

新規に論文リスト 'pubmed_論文リスト_3.csv' を作成します。
PubMedで検索中...
1件の論文IDを取得しました。詳細情報を1件ずつ処理します。

--- 新規論文を処理中 (PMID: 41017113) ---
要約を翻訳中...

1件の新規論文が見つかりました。CSVファイルに追加します。
'pubmed_論文リスト_3.csv' に保存しました。
